<a href="https://colab.research.google.com/github/Seoyeon-An/YBIGTA_Can_Classification/blob/model/YOLOv8_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

In [2]:
# check if selected GPU is available or use CPU
import torch
if torch.cuda.is_available():
    print('Good to go! You are using', torch.cuda.get_device_name())
    device = torch.device('cuda')
else:
    print('Please set GPU via Edit -> Notebook Settings.')
    device = torch.device('cpu')
     
print('Default torch.dtype is:', torch.get_default_dtype())

Good to go! You are using Tesla T4
Default torch.dtype is: torch.float32


## Constructing Dataset

In [3]:
#Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q /content/drive/MyDrive/ybigta/2023-1/Conference/Total_can_drink.zip -d /content/Dataset

In [ ]:
import os
import random
from sklearn.model_selection import train_test_split
import shutil

# Set the path to your image dataset folder
dataset_folder = "/content/Dataset/download"

# Set the path for the train and test folders
train_folder = "/content/CanDataset/train"
test_folder = "/content/CanDataset/test"
val_folder = "/content/CanDataset/val"

# Set the random seed for reproducibility
random.seed(42)

# Get the list of subfolders (image classes) in the dataset folder
subfolders = [f.path for f in os.scandir(dataset_folder) if f.is_dir()]

# Create train and test folders
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Iterate over each subfolder and split the images
for subfolder in subfolders:
    class_label = os.path.basename(subfolder)
    images = [f.path for f in os.scandir(subfolder) if f.is_file()]
    
    # Split the images into train and test sets
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
    train_images, val_images = train_test_split(train_images, test_size = 0.2, random_state = 42)
    
    # Create train and test folders for the class
    train_class_folder = os.path.join(train_folder, class_label)
    test_class_folder = os.path.join(test_folder, class_label)
    val_class_folder = os.path.join(val_folder, class_label)
    os.makedirs(train_class_folder, exist_ok=True)
    os.makedirs(test_class_folder, exist_ok=True)
    os.makedirs(val_class_folder, exist_ok=True)
    
    # Move the train images to the train folder
    for image in train_images:
        shutil.copy2(image, train_class_folder)
    
    # Move the test images to the test folder
    for image in test_images:
        shutil.copy2(image, test_class_folder)

    # Move the validation images to the val folder
    for image in test_images:
        shutil.copy2(image, val_class_folder)


In [ ]:
import yaml

data_dict = {
    "train": "/content/CanDataset/train",
    "val": "/content/CanDataset/val",
    "test": "/content/CanDataset/test",
}

with open("/content/CanDataset/data.yaml", 'w') as f:
  yaml.dump(data_dict, f)

In [ ]:
colab_folder = '/content/CanDataset'
drive_folder = '/content/drive/MyDrive/ybigta/2023-1/Conference/'  # Replace with the desired destination folder in Google Drive

zip_file = f'{drive_folder}/CanDataset'  # Replace with the desired name for the zip file

shutil.make_archive(zip_file, 'zip', colab_folder)

'/content/drive/MyDrive/ybigta/2023-1/Conference/CanDataset.zip'

In [4]:
!unzip -q /content/drive/MyDrive/ybigta/2023-1/Conference/CanDataset.zip -d /content/CanDataset

## Model

In [5]:
model = YOLO('yolov8n-cls.pt') 

model.train(data='/content/CanDataset/', epochs=1, imgsz=32)

100%|██████████| 5.28M/5.28M [00:00<00:00, 51.8MB/s]
Ultralytics YOLOv8.0.108 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/CanDataset/, epochs=1, patience=50, batch=16, imgsz=32, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=Fals

In [9]:
model.predict('/content/CanDataset/test/갈배사이다 캔/naver_0018.jpg')


image 1/1 /content/CanDataset/test/갈배사이다 캔/naver_0018.jpg: 32x32 갈아만든배 캔  0.07, 데자와 캔 0.07, 비락식혜 캔 0.06, 조지아 맥스 캔 0.05, 코카콜라 캔 0.04, 4.2ms
Speed: 1.0ms preprocess, 4.2ms inference, 0.1ms postprocess per image at shape (1, 3, 32, 32)


[ultralytics.yolo.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 keys: ['probs']
 masks: None
 names: {0: '갈배사이다 캔', 1: '갈아만든배 캔 ', 2: '게토레이 캔', 3: '구론산 캔', 4: '나랑드사이다 캔', 5: '데미소다 캔', 6: '데자와 캔', 7: '레쓰비 캔', 8: '맥스웰 캔', 9: '맥콜 캔', 10: '몬스터 캔', 11: '밀키스 캔', 12: '봉봉 캔', 13: '비락식혜 캔', 14: '삿뽀로 캔', 15: '솔의눈 캔', 16: '아사히 캔', 17: '암바사 캔', 18: '오라떼 캔', 19: '자몽소다 캔', 20: '조지아 맥스 캔', 21: '칠성사이다 캔', 22: '칭따오 캔', 23: '카스 캔', 24: '칸타타 캔', 25: '컨피던스 캔', 26: '코카콜라 캔', 27: '코코팜 캔', 28: '킨사이다 캔', 29: '테라 캔', 30: '토레타 캔', 31: '트로피카나 캔', 32: '파워에이드 캔', 33: '펩시 캔', 34: '포카리스웨트 캔', 35: '하이네켄 캔', 36: '하이트 캔', 37: '핫식스 캔', 38: '환타 캔'}
 orig_img: array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
     

In [10]:
model.export(format= 'onnx')

Ultralytics YOLOv8.0.108 🚀 Python-3.10.11 torch-2.0.1+cu118 CPU

PyTorch: starting from runs/classify/train/weights/best.pt with input shape (16, 3, 32, 32) BCHW and output shape(s) (16, 39) (2.9 MB)
requirements: Ultralytics requirement "onnx>=1.12.0" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 149.0 MB/s eta 0:00:00

requirements: 1 package updated per ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.14.0 opset 17...
ONNX: export success ✅ 15.7s, saved as runs/classify/train/weights/best.onnx (5.7 MB)

Export complete (15.9s)
Results saved to /content/runs/classify/train/weights
Predict:         yolo predict task=classify model=runs/classify/train/weights/best.onnx imgsz=32 
Validate:        yolo val task=classify model=runs/classify/train/weights/best.onnx 

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



'runs/classify/train/weights/best.onnx'